In [1]:
import sys
import os
import numpy as np

sys.path.append(os.path.join(os.pardir, "my_lib"))

In [2]:
from utils import softmax, cross_entropy_error, numerical_gradient, sigmoid

In [3]:
class simpleNet:    
    def __init__(self):
        self.W = np.random.randn(2, 3)
    
    def predict(self, x):
        return np.dot(x, self.W)

    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
    
        return loss

In [4]:
net = simpleNet()
print("Weight :\n", net.W)

x = np.array([0.6, 0.9])
p = net.predict(x)
print("Predict :", p)
print("Index of max value :", np.argmax(p))

t = np.array([0, 0, 1])
print("Loss :", net.loss(x, t))

Weight :
 [[-1.43070418 -0.1233304   0.14733817]
 [ 1.40904479  1.02171183  1.46729956]]
Predict : [0.4097178  0.84554241 1.40897251]
Index of max value : 2
Loss : 0.6613502032129314


In [5]:
dW = numerical_gradient(lambda w: net.loss(x, t), net.W)
print(dW)

[[ 0.11401435  0.17629328 -0.29030763]
 [ 0.17102153  0.26443992 -0.43546145]]


In [7]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, 
                 output_size, weight_init_std=0.01):
        self.params = {}
        self.params["w1"] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params["b1"] = np.zeros(hidden_size)
        self.params["w2"] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params["b2"] = np.zeros(output_size)
        
    def predict(self, x):
        w1, w2 = self.params["w1"], self.params["w2"]
        b1, b2 = self.params["b1"], self.params["b2"]
        a1 = np.dot(x, w1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, w2) + b2
        y = softmax(a2)
        return y
    
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y, t)
        
    def accuracy(slef, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)    
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_w = lambda w: self.loss(x, t)
        grads = {}
        grads["w1"] = numerical_gradient(loss_w, self.params["w1"])
        grads["b1"] = numerical_gradient(loss_w, self.params["b1"])
        grads["w2"] = numerical_gradient(loss_w, self.params["w2"])
        grads["b2"] = numerical_gradient(loss_w, self.params["b2"])
        return grads

In [9]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
print(net.params["w1"].shape)
print(net.params["b1"].shape)
print(net.params["w2"].shape)
print(net.params["b2"].shape)

(784, 100)
(100,)
(100, 10)
(10,)
